In [ ]:
#| default_exp process

# Multithreading

> API details.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import mmcv
from loguru import logger
def multi_thread(fn, array_inputs, max_workers=None, desc="Multi-thread Pipeline", unit="Samples", verbose=True, pbar_iterval=50):
    from concurrent.futures import ThreadPoolExecutor
    from functools import partial
    from glob import glob
    from multiprocessing import Pool
    from tqdm import tqdm

    def _wraper(x):
        i, input = x
        return {i: fn(input)}

    array_inputs = [(i, _) for i, _ in enumerate(array_inputs)]
    if verbose:
        progress_bar = tqdm(total=len(array_inputs))
    outputs = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i, result in enumerate(executor.map(_wraper, array_inputs)):
            outputs.update(result)
            if verbose:# and i%pbar_iterval==0:
                progress_bar.update()
    if verbose:
        logger.info('multi_thread')
    outputs = list(outputs.values())
    return outputs


In [ ]:
#|export
def multi_process(f, inputs, num_workers=10):
    from multiprocessing import Pool
    from tqdm import tqdm
    with Pool(10) as p:
        it = p.imap(f, inputs)
        total = len(inputs)
        return list(tqdm(it, total=total))


---

#### multi_process

>      multi_process (f, inputs, num_workers=10)

In [ ]:
#| hide
show_doc(multi_process)

---

#### multi_process

>      multi_process (f, inputs, num_workers=10)

## Example

In [ ]:
show_doc(multi_thread)

import time
import mmcv

def f(i):
    time.sleep(0.5)
    return i
timer = mmcv.Timer()

multi_thread(f, range(6), max_workers=2)
assert timer.since_start() < 2
print(f'Since start: {timer.since_start():0.2f} s')

multi_process(f, range(10))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.99it/s]

Since start: 1.51 s



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.21it/s]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
!nbdev_export

create_modules(nbname='/home/anhvth8/avcv/nbs/04_debug.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/01_process.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/05_coco_dataset.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/all.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/03_utils.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/00_visualize.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
create_modules(nbname='/home/anhvth8/avcv/nbs/index.ipynb', lib_path=Path('/home/anhvth8/avcv/avcv'), procs=[black_format])
